In [1]:
from tifffile import imread
import roifile
import glob
import os
import pathlib
import numpy as np
from PIL import Image
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import ConvexHull
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
from skimage import filters

In [34]:
DIRECTORY = r'D:\2D fibers alignments\FibersMAX\FibersMAX\KO'

# orientation and coherency are computed with OrientationJ imageJ plugin
orientation_folder = os.path.join(DIRECTORY, 'orientation')
coherency_folder = os.path.join(DIRECTORY, 'coherency')

In [ ]:
img_paths = glob.glob(DIRECTORY + '/*.tif')

data = []

for img_path in img_paths:
    
    folder_name = pathlib.PurePath(img_path).parents[0].name
    img_name = pathlib.PurePath(img_path).name.replace('.tif', '')
    
    # Open files
    orientation_path = os.path.join(orientation_folder, img_name + '_OJ.tif')
    coherency_path = os.path.join(coherency_folder, img_name + '_OJ.tif')
    
    mask_path = img_path.replace('.tif', '.png') #binary mask of fibers
    mask = Image.open(mask_path)
    mask = np.array(mask)/255

    circle_path = img_path.replace('.tif', '_ROI.tif.roi')
    circle_roi = roifile.ImagejRoi.fromfile(circle_path)
    circle_coordinates = circle_roi.coordinates()

    image = imread(img_path)
    orientation = imread(orientation_path)
    coherency = imread(coherency_path)

    # Get coordinates of pixels corresponding to fibers
    indices = np.where(mask == 1)
    mask_coordinates = np.array([ [x,y] for x,y in zip(indices[0], indices[1]) ]) 
    weights = [coherency[x,y] for x,y in zip(indices[0], indices[1])]
    
    # Find the closest point on the ROI
    distances = cdist(mask_coordinates, circle_coordinates)
    closest_point_indices = np.argmin(distances, axis=1)
    closest_points = circle_coordinates[closest_point_indices]
    min_dist = [np.min(distances[i,:]) for i in range(distances.shape[0])]

    #  Compute fiber alignment to the closest tangent from the ROI
    norm_vectors = normal_vectors(circle_coordinates)
    angle_diffs = np.zeros(len(mask_coordinates))

    for i in range(len(mask_coordinates)):
        
        vector_angle = orientation[mask_coordinates[i,0], mask_coordinates[i,1]]
        angle_rad = np.deg2rad(vector_angle)
        direction_vector = np.array([np.sin(angle_rad), np.cos(angle_rad)])
        
        closest_point = closest_points[i]
        idx = np.where((circle_coordinates == closest_point).all(axis=1))[0][0]
        normv = norm_vectors[idx]
        angle_diffs[i] = np.dot(direction_vector,normv)
        

    df = pd.DataFrame({
                        'angle': angle_diffs , 
                        'dist2circle': min_dist,
                        'coherency': weights
    })
    
    df['image'] = [img_name]*len(df)
    df['folder'] = [folder_name]*len(df)
    
    data.append(df)
    
    
# Export data 
data = pd.concat(data)